#### Reload images and videos datasets

In [1]:
import pandas as pd
import json

df_images = pd.read_csv('data/datasets/images.csv')
df_videos = pd.read_csv('data/datasets/videos.csv')

# df_images['tags'] = df_images['tags'].apply(json.loads)
df_videos['tags'] = df_videos['tags'].apply(json.loads)

print(df_videos.columns)
print()
display(df_videos.index[:10])

Index(['_id', 'blob_name', 'blob_size', 'bucket_name', 'file_name', 'code',
       'n_folders', 'timestamp', 'folder_structure', 'folder', 'tags', 'url',
       'api_url', 'bucket', 'seen'],
      dtype='object')



RangeIndex(start=0, stop=10, step=1)

#### Create tag field

In [2]:
from modules.octa_video_util import _assign_tag

# Create unique tag column based on class priority list
default_tag = 'normal'
tags_priority_list = ['alagamento', 'bolsão', 'lâmina', 'poça', 'transbordo']

df_videos['tag'] = df_videos['tags'].apply(lambda tags_list: _assign_tag(tags_list, tags_priority_list, default_tag))
df_images['tag'] = df_images['tags'].apply(lambda tags_list: _assign_tag(tags_list, tags_priority_list, default_tag))

display(df_videos.tag.value_counts().rename('Videos'))
print()
display(df_images.tag.value_counts().rename('Images'))

tag
normal        59695
poça           1744
bolsão          277
lâmina          153
alagamento       86
transbordo       62
Name: Videos, dtype: int64

tag
normal        154843
poça           69839
bolsão         10621
lâmina          5218
alagamento      5026
transbordo      2237
Name: Images, dtype: int64

#### Binarize 'tag' variable for images dataset

In [3]:
target_classes = ['lâmina', 'bolsão', 'alagamento']

# Binarize categorical variable from list of target classes
df_images['flood'] = df_images['tag'].isin(target_classes).astype(int)

display(df_images['flood'].value_counts())
print()
display(df_images.index[:10])

flood
0    226919
1     20865
Name: count, dtype: int64

RangeIndex(start=0, stop=10, step=1)

#### Filter videos

In [4]:
from modules.octa_video_util import filter_by_query

query_params = {
    'seen': True,
    'tag': ['normal', 'poça', 'lâmina', 'bolsão', 'alagamento']
}

# Filter dataset of images by query
df_videos_filtered = filter_by_query(df_videos, query_params).copy()

display(df_videos_filtered['tag'].value_counts())
print()
display(df_videos_filtered.index[:10])

tag
normal        4089
poça          1743
bolsão         275
lâmina         152
alagamento      85
Name: count, dtype: int64

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

#### Reload sample images dataset

In [5]:
# target_directory = 'data/splits/sgkf-8-1-1'
target_directory = 'data/splits/sgkf-6-2-2-size-2024-rs-3'

df_sample = pd.read_csv(f'{target_directory}/images.csv', index_col=0)

print(df_sample.columns)
display(df_sample.index[:10])

Index(['id_video', 'code', 'folder', 'file_name', 'file_path', 'frame_index',
       'timestamp', 'initial_timestamp', 'seen', 'tags', 'tag', 'flood',
       'set'],
      dtype='object')


Index([106170, 173236, 223883, 223535, 180913, 177444, 75080, 127184, 173623,
       225050],
      dtype='int64')

#### Unique cameras in each data set

In [6]:
train = df_sample[df_sample['set']=='train']
test = df_sample[df_sample['set']=='test']
val = df_sample[df_sample['set']=='val']

train_codes = train['code'].unique()
test_codes = test['code'].unique()
val_codes = val['code'].unique()
out_train_codes = set(df_videos_filtered['code'].unique()).difference(train_codes)

print('Train codes:', len(train_codes))
print('Test codes:', len(test_codes))
print('Val codes:', len(val_codes))
print('Out-train codes:', len(out_train_codes))


Train codes: 18
Test codes: 6
Val codes: 6
Out-train codes: 182


#### Test videos from cameras outside the training set

In [7]:
df_videos_test = df_videos_filtered[~df_videos_filtered['code'].isin(train_codes)]

display(df_videos_test['tag'].value_counts())
print()
display(df_videos_test.index[:10])
print()
print('Shape of sample for testing:', df_videos_test.shape)

tag
normal        3413
poça          1490
bolsão         221
lâmina         111
alagamento      47
Name: count, dtype: int64

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')


Shape of sample for testing: (5282, 16)


#### Undersample videos based on flood event type

In [8]:
n_majority_videos = 50
n_minority_videos = 50
random_state = 0

from imblearn.under_sampling import RandomUnderSampler

target_classes = ['lâmina', 'bolsão', 'alagamento']
test_minority = df_videos_test[df_videos_test['tag'].isin(target_classes)]
test_majority = df_videos_test.drop(test_minority.index)

x = test_majority.copy()
y = test_majority['tag'].copy()

rus = RandomUnderSampler(sampling_strategy='auto', random_state=random_state, replacement=False)

test_majority_filtered, tags_test_majority_filtered = rus.fit_resample(x, y)
test_majority_filtered = test_majority_filtered.sample(n_majority_videos, replace=False, random_state=random_state)
test_minority_filtered = test_minority.sample(n_minority_videos, replace=False, random_state=random_state)

df_videos_test_filtered = pd.concat([test_minority_filtered, test_majority_filtered])

print('Videos selected:', df_videos_test_filtered.shape)
print()
display(df_videos_test_filtered['tag'].value_counts())
print()
display(df_videos_test_filtered['code'].value_counts().to_frame().T)
print()
display(df_videos_test_filtered.index[:10])


Videos selected: (20, 16)



tag
bolsão        6
normal        5
poça          5
lâmina        3
alagamento    1
Name: count, dtype: int64

code,278.0,1538.0,298.0,97.0,1671.0,869.0,1460.0,1123.0,1639.0,2166.0,58.0,65.0,1649.0
count,4,3,2,2,1,1,1,1,1,1,1,1,1


Index([29905, 40993, 40844, 40839, 37897, 25468, 42353, 24952, 32861, 24230], dtype='int64')

#### Extract corresponding rows from images dataset

In [9]:
df_images_sample = df_images[df_images['id_video'].isin(df_videos_test_filtered['_id'])]

print('Images selected:', df_images_sample.shape)
print()
display(df_images_sample['tag'].value_counts())
print()
display(df_images_sample['code'].value_counts().to_frame().T)
print()
print('Videos found:', len(df_images_sample['id_video'].unique()))

Images selected: (764, 12)



tag
poça          225
bolsão        200
normal        159
lâmina        135
alagamento     45
Name: count, dtype: int64

code,1538.0,278.0,97.0,298.0,1123.0,58.0,1671.0,869.0,2166.0,1460.0,1639.0,1649.0,65.0
count,135,110,90,90,45,45,45,45,45,45,45,17,7



Videos found: 20


#### Check existence of images

In [10]:
import os

def get_nested_files(folder_path):
    file_paths = []
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path.replace('\\', '/'))
    
    return file_paths

# Example usage
# base_path = 'data/images'
# images_paths_found = get_nested_files(base_path)


base_path = 'data/images'

images_paths_found = get_nested_files(base_path)
images_paths_sample = df_images_sample['file_path'].apply(lambda file_path: f'{base_path}/{file_path}'.replace('\\', '/'))
files_exist_prct = images_paths_sample.isin(images_paths_found).mean()

str(round(files_exist_prct * 100, 2)) + ' %'

'100.0 %'

---

## Run predictions with YOLO

#### Load model with Yolo

In [11]:
from ultralytics import YOLO

# Path to the folder you want to zip
# model_path = f'models/sgkf-8-1-1/weights/best.pt'
model_path = f'models/sgkf-50-25-25-size-2024-rs-2/weights/best.pt'

# Load a model
model = YOLO(model_path)  # load a partially trained model

#### Run predictions with yolo in batch

In [12]:
import cv2
from IPython.display import clear_output as co
import time

base_path = 'data/images'
batch = 12

eval_imgs_df = df_images_sample.sort_values(['id_video', 'frame_index'])# .head(45)
img_path_list = f'{base_path}/' + eval_imgs_df['file_path']
n_imgs = len(img_path_list)

preds = []
avg_time = 0.0
s_time = time.time()
for i in range(0, n_imgs, batch):
    e_time = time.time() - s_time
    e_time_round = round(e_time / 60, 2)
    avg_time = e_time / max(1, i)
    expected_finish_time = round((n_imgs - i) * avg_time / 60, 2)
    expected_total_time = round(n_imgs * avg_time  / 60, 2)

    co(True)
    print(f'{i}/{n_imgs} | {e_time_round} min / {expected_total_time} min | time-left: {expected_finish_time} min')    

    img_path_list_batch = img_path_list[i: i + batch].tolist()
    pred = model.predict(img_path_list_batch, imgsz=640)
    pred = [[pred_i.probs.data[1].item(), pred_i.probs.top1] for pred_i in pred]
    preds.extend(pred)

eval_imgs_df[['prob', 'pred']] = preds

15540/15543 | 172.36 min / 172.4 min | time-left: 0.03 min

0: 640x640 0 0.94, 1 0.06, 1: 640x640 0 0.89, 1 0.11, 2: 640x640 0 0.89, 1 0.11, 2212.3ms
Speed: 45.0ms preprocess, 737.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


---

## Run predictions with PyTorch

#### Load model with PyTorch

In [1]:
from PIL import Image
import time; s = time.time()
import torch
import torch.nn.functional as F
from modules.models_util import ViT

weights_path = '../flood-vision/models/vit_b16_2024-02-11 07-01-38_epoch_20.pth'

to_device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(to_device)

model = ViT(device=to_device).load(weights_path.replace('\\', '/'))
model.to(device)
model.eval()

print(f'\nModel load time: {round(time.time() - s, 2)} secs')

All parameters for model VisionTransformer requires grad.
Weights for model VisionTransformer loaded from ../flood-vision/models/vit_b16_2024-02-11 07-01-38_epoch_20.pth

Model load time: 37.87 secs


#### PyTorch model utility functions

In [12]:
from torch.utils.data import DataLoader, Dataset

def pytorch_predict(batch):
    with torch.no_grad(): # no backtracking
        output = model(batch)
    
    probs = F.softmax(output, dim=1)
    return probs

class CustomDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

#### Run predictions with PyTorch in batches

In [13]:
import time
from IPython.display import clear_output as co
from modules.models_util import ViT

base_path = 'data/images'
batch_size = 8

eval_imgs_df = df_images_sample.sort_values(['id_video', 'frame_index']) # .head(45)
img_path_list = f'{base_path}/' + eval_imgs_df['file_path']
n_imgs = len(img_path_list)

# PyTorch transform
transform = ViT.data_transforms['test']

# Load images using the custom dataset
dataset = CustomDataset(list(img_path_list), transform=transform)

# Create a DataLoader for batching and shuffling (if needed)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0) # num_workers different from zero didnt work (got stucked)

# Iterate through the data loader
preds = []
avg_time = 0.0
i = 0
s_time = time.time()
for batch in data_loader:
    e_time = time.time() - s_time
    e_time_round = round(e_time / 60, 2)
    avg_time = e_time / max(1, i)
    expected_finish_time = round((n_imgs - i) * avg_time / 60, 2)
    expected_total_time = round(n_imgs * avg_time  / 60, 2)

    co(True)
    print(f'{i}/{n_imgs} | {e_time_round} min / {expected_total_time} min | time-left: {expected_finish_time} min')    
    
    probs = pytorch_predict(batch)
    pred = [[prob.item(), int(prob >= 0.5)] for prob in probs[:, 1]]
    preds.extend(pred)
    i += len(batch)
    
eval_imgs_df[['prob', 'pred']] = preds
# preds

95/764 | 23.11 min / 185.85 min | time-left: 162.74 min


---

#### Save or reload predictions

In [14]:
# target_directory = 'data/splits/sgkf-8-1-1'
# prediction_samples_path = f'{target_directory}/images--cameras-yolo.csv'
prediction_samples_path = f'{target_directory}/images-out-cameras-pytorch.csv'

eval_imgs_df.to_csv(prediction_samples_path, index=True)

# eval_imgs_df = pd.read_csv(prediction_samples_path, index_col=0)

#### Evaluate results

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, jaccard_score

labels = eval_imgs_df['flood'].tolist()
pred_labels = eval_imgs_df['pred'].tolist()
# pred_labels = [pred[0] for pred in preds]

print('\n * Confusion Matrix:')
print(confusion_matrix(labels, pred_labels))
print('\n * Classification Report:')
print(classification_report(labels, pred_labels))
print('\n * Jaccard Score:')
print(jaccard_score(labels, pred_labels))



 * Confusion Matrix:
[[216 168]
 [238 142]]

 * Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.56      0.52       384
           1       0.46      0.37      0.41       380

    accuracy                           0.47       764
   macro avg       0.47      0.47      0.46       764
weighted avg       0.47      0.47      0.46       764


 * Jaccard Score:
0.2591240875912409


#### Evaluation per camera

In [16]:
import numpy as np
from sklearn.metrics import jaccard_score

for code in eval_imgs_df['code'].value_counts().index:
    code_msk = eval_imgs_df['code'] == code
    code_labels = np.array(labels)[code_msk]
    code_pred_labels = np.array(pred_labels)[code_msk]

    print('\n * Confusion_matrix:')
    print(confusion_matrix(code_labels, code_pred_labels))
    print('\n * Classification_report:')
    print(classification_report(code_labels, code_pred_labels))
    print('\n * Jaccard Score:')
    print(jaccard_score(code_labels, code_pred_labels))



 * Confusion_matrix:
[[ 0  0]
 [92 43]]

 * Classification_report:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.32      0.48       135

    accuracy                           0.32       135
   macro avg       0.50      0.16      0.24       135
weighted avg       1.00      0.32      0.48       135


 * Jaccard Score:
0.31851851851851853

 * Confusion_matrix:
[[ 0  0]
 [65 45]]

 * Classification_report:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.41      0.58       110

    accuracy                           0.41       110
   macro avg       0.50      0.20      0.29       110
weighted avg       1.00      0.41      0.58       110


 * Jaccard Score:
0.4090909090909091

 * Confusion_matrix:
[[53 37]
 [ 0  0]]

 * Classification_report:
              precision    recall  f1-score   support

         0

C:\Users\luisr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luisr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luisr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luisr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1

0.0

 * Confusion_matrix:
[[22 23]
 [ 0  0]]

 * Classification_report:
              precision    recall  f1-score   support

         0.0       1.00      0.49      0.66        45
         1.0       0.00      0.00      0.00         0

    accuracy                           0.49        45
   macro avg       0.50      0.24      0.33        45
weighted avg       1.00      0.49      0.66        45


 * Jaccard Score:
0.0

 * Confusion_matrix:
[[24 21]
 [ 0  0]]

 * Classification_report:
              precision    recall  f1-score   support

         0.0       1.00      0.53      0.70        45
         1.0       0.00      0.00      0.00         0

    accuracy                           0.53        45
   macro avg       0.50      0.27      0.35        45
weighted avg       1.00      0.53      0.70        45


 * Jaccard Score:
0.0

 * Confusion_matrix:
[[24 21]
 [ 0  0]]

 * Classification_report:
              precision    recall  f1-score   support

         0.0       1.00      0.53    

C:\Users\luisr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luisr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luisr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luisr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1

#### Function to write video results

In [17]:
import os
import cv2

def create_annotated_video(image_paths, true_labels, predicted_labels, predicted_probs, output_video_path, report=True):
    if len(image_paths) != len(true_labels) or len(true_labels) != len(predicted_labels):
        raise ValueError("Number of paths, true labels, and predicted labels must be the same.")

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, 3.0, (854, 480)) # (640, 480)

    n_imgs = len(image_paths)
    for i, (image_path, true_label, predicted_label, predicted_prob) in enumerate(zip(image_paths, true_labels, predicted_labels, predicted_probs)):
        # Read the image
        image = cv2.imread(image_path)

        # Set color based on prediction correctness
        if true_label == predicted_label:
            color = (0, 255, 0)  # Green for correct predictions
        else:
            color = (0, 0, 255)  # Red for incorrect predictions

        # Annotate the image with true and predicted labels in the bottom-left corner
        true_label_text = f'True: {true_label}'
        predicted_label_text = f'Predicted: {int(predicted_label)}'
        probability_text = f'Probability: {round(predicted_prob * 100, 1)} %'
        cv2.putText(image, true_label_text, (image.shape[1] - 130, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        cv2.putText(image, predicted_label_text, (image.shape[1] - 210, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        cv2.putText(image, probability_text, (image.shape[1] - 315, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        # Resize the image for video (optional)
        image = cv2.resize(image, (854, 480)) # (640, 480)

        # Write the annotated image to the video
        video_writer.write(image)

        # Report progress
        if report:
            print(f'Images processed: {i + 1}/{n_imgs} ', end='\r')    

    # Release the VideoWriter
    video_writer.release()

# Example usage:

# base_path = 'data/images'

# unique_video_ids = eval_imgs_df['id_video'].unique()
# video_imgs = eval_imgs_df[eval_imgs_df['id_video']==unique_video_ids[0]]

# image_paths = (f'{base_path}/' + video_imgs['file_path']).tolist()
# true_labels = video_imgs['flood'].tolist()
# predicted_labels = video_imgs['pred'].tolist()
# predicted_probs = video_imgs['prob'].tolist()

# output_video_path = 'output_video.mp4'

# create_annotated_video(image_paths, true_labels, predicted_labels, predicted_probs, output_video_path)


#### Write video results

In [18]:
import os

base_path = 'data/images'
# base_output_path = 'data/eval-videos-yolo'
base_output_path = 'data/eval-videos-pytorch'

unique_camera_codes = eval_imgs_df['code'].unique()
n_cameras = len(unique_camera_codes)

for i, code in enumerate(unique_camera_codes):
    for true_label in [0, 1]:
        camera_imgs = eval_imgs_df[(eval_imgs_df['code'] == code) & (eval_imgs_df['flood'] == true_label)].sort_values('timestamp')

        if not len(camera_imgs):
            continue

        image_paths = (f'{base_path}/' + camera_imgs['file_path']).tolist()
        true_labels = camera_imgs['flood'].tolist()
        predicted_labels = camera_imgs['pred'].tolist()
        predicted_probs = camera_imgs['prob'].tolist()
        output_video_path = f'{base_output_path}/{true_label}/{int(code)}.mp4'
        
        output_video_dir = os.path.dirname(output_video_path)
        if not os.path.isdir(output_video_dir):
            os.makedirs(output_video_dir)
            
        create_annotated_video(image_paths, true_labels, predicted_labels, predicted_probs, output_video_path, report=False)
        print(f'Cameras processed: {i + 1}/{n_cameras} ', end='\r')    
        

#### End

---

### Extra:

#### Load model with TensorFlow

In [ ]:
# Load the best model from the saved checkpoint file
checkpoint_filepath = 'results/best_model_cnn.h5'
model1 = models.load_model(checkpoint_filepath)

params = {
    'data': 'data/images',
    'epochs': 25,
    'imgsz': 640,
    'batch': 32,
    'device': [0, 1],
    'learning_rate': 0.0001,
}

img_height, img_width = 640, 640 # 480, 854

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    params['data'] + '/test',
    target_size=(224, 224),
    batch_size=params['batch'],
    class_mode='binary',
    shuffle=False,
)


#### Copy images into test folder

In [12]:
from modules.octa_video_util import copy_images_to_folders

base_directory = 'data/images'
target_directory = 'data/splits/sgkf-8-1-1-test-videos'

dataset = df_images_sample.copy()
file_path_field = 'file_path'
label_field = 'flood'

train_indexes = None
test_indexes = list(df_images_sample.index)
val_indexes = None

copy_images_to_folders(
    base_directory, target_directory, dataset,
    train_indexes, test_indexes, val_indexes,
    file_path_field=file_path_field, tag_field=label_field
)


Copying images to test folders:


#### Save dataframe of sample images

In [59]:
target_directory = 'data/splits/sgkf-8-1-1-videos'

dataset = df_images_sample.copy()

# data_train = dataset.loc[Y_train.index]
data_test = dataset.loc[test_indexes]
# data_val = dataset.loc[Y_val.index]

# data_train['set'] = 'train'
data_test['set'] = 'test'
# data_val['set'] = 'val'

data_split_df = data_test.copy()
# data_split_df = pd.concat([data_train, data_test, data_val])

data_split_df.to_csv(f'{target_directory}/images.csv')
print(f'split dataframe saved with shape: {data_split_df.shape}')

split dataframe saved with shape: (1070, 13)


#### Count save images

In [61]:
import os

target_directory = 'data/splits/sgkf-8-1-1-videos'

# print('train:', len(os.listdir(f'{target_directory}/train/0')), len(os.listdir(f'{target_directory}/train/1')))
print('test:', len(os.listdir(f'{target_directory}/test/0')), len(os.listdir(f'{target_directory}/test/1')))
# print('val:', len(os.listdir(f'{target_directory}/val/0')), len(os.listdir(f'{target_directory}/val/1')))

test: 247 823


#### Set up TensorFlow

In [64]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available.")
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    print("GPU not found. Using CPU.")
    
# Define mirrored strategy for GPU training
strategy = tf.distribute.MirroredStrategy()

GPU not found. Using CPU.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
